<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
import time
import numpy as np
import types
import os
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from scipy.special import zeta
import PRyM.PRyM_init as PRyMini
import PRyM.PRyM_thermo as PRyMthermo
import PRyM.test_main3 as test_main
#import PRyM.PRyM_jl_sys as PRyMjl
from numdifftools import Derivative
import PRyM.PRyM_eval_nTOp as PRyMevalnTOp
import PRyM.PRyM_nTOp as PRyMnTOp
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
#observed values, all wrong currently, for testing purposes
#Neff_obs, Omeganurel_obs, OneOverOmeganunr_obs, YPCMB_obs, YPBBN_obs, DoHx1e5_obs, He3oHx1e5_obs, Li7oHx1e10_obs = np.array([3.04439907, 5.69942692, 93.03336616, 0.24552507, 0.24685084, 2.5276775, 1.04572838, 5.27256451])
#Yp, Omega_B h^2, Neff
#YPCMB_obs, OmegaB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs
YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs = np.array([0.24,3.85,2.547,1.63])
X_obs = np.array([YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs])
#X_obs = np.array([Neff_obs, Omeganurel_obs, OneOverOmeganunr_obs, YPCMB_obs, YPBBN_obs, DoHx1e5_obs, He3oHx1e5_obs, Li7oHx1e10_obs])
#Neff_sig, Omeganurel_sig, OneOverOmeganunr_sig, YPCMB, YPBBN_sig, DoHx1e5_sig, He3oHx1e5_sig, Li7oHx1e10_sig = np.array([0.5,0.5,3,.1,.1,.3,.2,1])
YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig = np.array([0.24,0.62,0.029,0.3])
#sigma_X = np.array([Neff_sig, Omeganurel_sig, OneOverOmeganunr_sig, YPCMB, YPBBN_sig, DoHx1e5_sig, He3oHx1e5_sig, Li7oHx1e10_sig])
sigma_X = np.array([YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig])

In [3]:
#chi2_fn(params)

In [4]:
#params

In [5]:
filename = "output.txt"

# Create the file (or clear it if it already exists)
with open(filename, 'w') as f:
    f.write("Starting output:\n")

In [6]:
# Chi^2 function
#params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]
def chi2_fn(params):
    #initialize stasis parameters
    PRyMini.compute_bckg_flag = True
    PRyMini.save_bckg_flag = False
    PRyMini.stasis_flag = True
    PRyMini.verbose_flag = False
    PRyMini.stasis_params['gamma'] = params[0]
    PRyMini.stasis_params['delta'] = params[1]
    PRyMini.stasis_params['alpha'] = params[2]
    PRyMini.stasis_params['Delta_m'] = params[3]
    PRyMini.stasis_params['m0'] = params[4]
    PRyMini.stasis_params['plasma_inject_frac'] = params[5]
    PRyMini.stasis_params['tiny_radiation'] = params[6]
    PRyMini.stasis_params['stasis_end_mev'] = params[7]
    PRyMini.stasis_params['N_SPECIES'] = int(params[8])
    print(PRyMini.stasis_params)
    #predictions = model(params)
    #residuals = (predictions - X_obs) / errors
    #sum(((X_th-X_obs)/sigma_X)**2)
    #calculate theoretical values
    results = test_main.PRyMclass().PRyMresults()
    Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10 = results
    #X_th = np.array([Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10])
    X_th = np.array([YPCMB, Neff, DoHx1e5, Li7oHx1e10])
    return sum(((X_th-X_obs)/sigma_X)**2)

# Gradient of chi^2 (numerical)
def grad_chi2(params, eps=1e-5):
    grad = np.zeros_like(params)
    for i in range(len(params)):
        dparams = np.zeros_like(params)
        dparams[i] = eps
        grad[i] = (chi2_fn(params + dparams) - chi2_fn(params - dparams)) / (2 * eps)
    return grad

# Gradient descent routine
def gradient_descent(init_params, lr=0.01, tol=1e-6, max_iter=100):
    params = np.array(init_params)#, dtype=float)
    with open(filename, 'a') as f:
        f.write(str(params))
        f.flush()
    for step in range(max_iter):
        print(step)
        grad = grad_chi2(params)
        params -= lr * grad
        with open(filename, 'a') as f:
            f.write(str(params))
            f.flush()

        if np.linalg.norm(grad) < tol:
            print(f"Converged in {step} steps.")
            break
    return params

In [7]:
PRyMini.stasis_params

{'gamma': 7,
 'delta': 1,
 'm0': 1.0,
 'Delta_m': 1.0,
 'N_SPECIES': 10,
 'GAMMA_OVER_H': 100000.0,
 'stasis_end_mev': 50,
 'tiny_radiation': 0.01,
 'plasma_inject_frac': 0.1,
 'alpha': 1.0}

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:
#initialize random stasis parameters in the desired intervals
gamma = random.uniform(4,8)
delta = random.uniform(0,3)
alpha = (2/7)*gamma - (1/delta)
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = random.uniform(0.1,1)
Tend = random.uniform(1, 10)
Nspecies = int(random.randint(1, 20))
#PRyMini.stasis_params
print(Nspecies)
#initialize params
params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]

#apply gradient descent, extremely slow right now
gradient_descent(params, lr=0.01, tol=1e-6, max_iter=100)

20
0
{'gamma': np.float64(5.809355705496223), 'delta': np.float64(2.9811026795260283), 'm0': np.float64(5.703363042766905), 'Delta_m': np.float64(5.703363042766905), 'N_SPECIES': 20, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(5.628032816623824), 'tiny_radiation': np.float64(0.2832358299160309), 'plasma_inject_frac': np.float64(0.5464390034666677), 'alpha': np.float64(1.3243667130143755)}


{'gamma': np.float64(5.809335705496224), 'delta': np.float64(2.9811026795260283), 'm0': np.float64(5.703363042766905), 'Delta_m': np.float64(5.703363042766905), 'N_SPECIES': 20, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(5.628032816623824), 'tiny_radiation': np.float64(0.2832358299160309), 'plasma_inject_frac': np.float64(0.5464390034666677), 'alpha': np.float64(1.3243667130143755)}


{'gamma': np.float64(5.809345705496224), 'delta': np.float64(2.9811126795260283), 'm0': np.float64(5.703363042766905), 'Delta_m': np.float64(5.703363042766905), 'N_SPECIES': 20, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(5.628032816623824), 'tiny_radiation': np.float64(0.2832358299160309), 'plasma_inject_frac': np.float64(0.5464390034666677), 'alpha': np.float64(1.3243667130143755)}


{'gamma': np.float64(5.809345705496224), 'delta': np.float64(2.981092679526028), 'm0': np.float64(5.703363042766905), 'Delta_m': np.float64(5.703363042766905), 'N_SPECIES': 20, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(5.628032816623824), 'tiny_radiation': np.float64(0.2832358299160309), 'plasma_inject_frac': np.float64(0.5464390034666677), 'alpha': np.float64(1.3243667130143755)}


{'gamma': np.float64(5.809345705496224), 'delta': np.float64(2.9811026795260283), 'm0': np.float64(5.703363042766905), 'Delta_m': np.float64(5.703363042766905), 'N_SPECIES': 20, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(5.628032816623824), 'tiny_radiation': np.float64(0.2832358299160309), 'plasma_inject_frac': np.float64(0.5464390034666677), 'alpha': np.float64(1.3243767130143755)}


...

{'gamma': np.float64(5.809345705496224), 'delta': np.float64(2.9811026795260283), 'm0': np.float64(5.703363042766905), 'Delta_m': np.float64(5.703363042766905), 'N_SPECIES': 20, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(5.628032816623824), 'tiny_radiation': np.float64(0.2832358299160309), 'plasma_inject_frac': np.float64(0.5464390034666677), 'alpha': np.float64(1.3243667130143755)}


{'gamma': np.float64(5.809345705496224), 'delta': np.float64(2.9811026795260283), 'm0': np.float64(5.703363042766905), 'Delta_m': np.float64(5.703363042766905), 'N_SPECIES': 19, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(5.628032816623824), 'tiny_radiation': np.float64(0.2832358299160309), 'plasma_inject_frac': np.float64(0.5464390034666677), 'alpha': np.float64(1.3243667130143755)}


1
{'gamma': np.float64(-391.06532448511024), 'delta': np.float64(2.9811026795260283), 'm0': np.float64(242.6817298973675), 'Delta_m': np.float64(319.10262429232444), 'N_SPECIES': 4, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(-414.6348424781578), 'tiny_radiation': np.float64(-277.3000089475586), 'plasma_inject_frac': np.float64(-44.26126895503097), 'alpha': np.float64(508.668848270155)}


/dfs6/pub/ahewitt1/Stasis/Stasis_prym/PRyM/test_main3.py:749: RuntimeWarning: overflow encountered in power
  omega = (m_ell/m0)**alpha
/dfs6/pub/ahewitt1/Stasis/Stasis_prym/PRyM/test_main3.py:750: RuntimeWarning: invalid value encountered in divide
  omega /= omega.sum()
/dfs6/pub/ahewitt1/Stasis/Stasis_prym/PRyM/test_main3.py:782: RuntimeWarning: divide by zero encountered in log
  "chi_ell0":   np.log(rho_ell0),


ValueError: All components of the initial state `y0` must be finite.